In [ ]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz
import plotly.express as px
import os

from pathlib import Path 
import time

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/testing_data_all_20220523.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Columns: 299 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(4), object(5)
memory usage: 1.1+ MB


In [ ]:
test.head(2)

,Unnamed: 0,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,DRAIN_SQKM,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,0,2021,11,0.000,22595619,mean,observed,8.0,18.0,57.7287,...,0.131168,2.213277,1.077336,1.245494,62.235627,42.775919,31.027255,3.117207,2.139399,1.551621
1,1,2021,11,0.351,22591931,mean,observed,8.0,18.0,10.8657,...,1.677609,5.268198,1.707801,1.453772,77.353606,45.856562,44.561237,4.341184,2.345440,2.250610


In [ ]:
#right now we have Nov 2021 - Apr 2022 data
test['month'].unique()

array([11, 12,  1,  2,  3,  4])

In [ ]:
#no null columns
test = test.dropna(axis=0,how='any')
test.isnull().sum().sum()

0

In [ ]:
test['flow_cfs'] = test['flow_cfs'] / test['DRAIN_SQKM']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','Unnamed: 0'] 
df = test.drop(columns = col)
#label encoding
cols_encoding = ['NewEco3']
labelencoder = LabelEncoder()
df['NewEco3_label'] = labelencoder.fit_transform(df['NewEco3'])
df = df.drop(columns=cols_encoding)
df.reset_index(drop=True, inplace=True)

In [ ]:
def x_test(month):  
  test1 = df[df['month']==month]
  x_test = test1.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  test1 = df[df['month']==month]
  y_test = test1.flow_cfs.values
  return y_test

In [ ]:
nov_data = x_test(11)
nov_data.take([0])

,month,DRAIN_SQKM,SLOPE_PCT_30M,ELEV_MEAN_M_BASIN_30M,ELEV_MIN_M_BASIN_30M,ELEV_MAX_M_BASIN_30M,ElvRng,TOT_ECOL3_1,TOT_ECOL3_2,TOT_ECOL3_3,...,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum,NewEco3_label
0,11,57.7287,44.82,1339.49,317.8,1952.06,1634.26,0.0,0.0,0.0,...,2.213277,1.077336,1.245494,62.235627,42.775919,31.027255,3.117207,2.139399,1.551621,2


In [ ]:
# DONE - [Kirk] Let's convert this to a function so we can loop through all the months
# First task is to read in all the file names in the two results folders, parse the text of the file name to get the month, and then select either the most recent file for each month or the one with the best r2.
# Next we need a way to reduce the input data based on the list of column names
# Finally, I would like a summary dataframe and csv that lists out each month, the R2 of the full and reduced models, and the file name of the full and reduced model

In [ ]:
def testing_full_model(month):
    directory = '/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model'
    r2_list = []
    max_r2 = None

    for filename in os.listdir(directory):
        if filename.endswith(".pkl"):
          fn = int(filename.split("mymodel_")[1].split("_")[0])
          if fn == month:
            r2 = filename.split("r2_")[1].split("_")[0]
            r2_list.append(r2)
            for each_r2 in r2_list:
              if (max_r2 is None or each_r2 > max_r2):
                max_r2 = each_r2
          else:
            continue
          if max_r2 in filename:
            print(filename)
            pickled_model = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/'+filename, 'rb'))
            print(pickled_model)

    predictions = pickled_model.predict(x_test(month)) 
    r2_test = r2_score(y_test(month)+1, predictions+1) 
    print('Full model r-squared scores:', round(r2_test, 2))

# def testing_reduced_model(month):
#     directory = '/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features'
#     r2_list = []
#     max_r2 = None

#     for filename in os.listdir(directory):
#         if filename.endswith(".pkl"):
#           fn = int(filename.split("mymodel_reduced_")[1].split("_")[0])
#           if fn == month:
#             r2 = filename.split("r2_")[1].split("_")[0]
#             r2_list.append(r2)
#             for each_r2 in r2_list:
#               if (max_r2 is None or each_r2 > max_r2):
#                 max_r2 = each_r2
#           else:
#             continue
#           if max_r2 in filename:
#             print(filename)
#             pickled_model_r = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/'+filename, 'rb'))
#             print(pickled_model_r)
#             rf_var = pickled_model_r.feature_names_in_
#             print(rf_var)
#             reduced_col_11 = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
#             df = test.loc[:, reduced_col_11]
#             print(df.head(2))
#             rf_r = pickled_model_r
#             predictions = rf_r.predict(x_test(month)) 
#             r2_test = r2_score(y_test(month)+1, predictions+1) 
#             print('R-squared scores:', round(r2_test, 4))

def compare_plot(df, x, y):
  """Create a scatter plot with a 1 to 1 dashed line for comparison of data"""
  fig = px.scatter(df, x=x, y=y)
  axes_max = max(df[x].max(), df[y].max())
  fig.update_yaxes(range=[0, axes_max])
  fig.update_xaxes(range=[0, axes_max])
  fig.update_layout(width=600, height=600)
  fig.add_shape(type='line',
                x0=0,
                y0=0,
                x1=axes_max,
                y1=axes_max,
                line=dict(color='Grey',dash='dash'),
                xref='x',
                yref='y'
  )
  return fig

November 2021

In [ ]:
testing_full_model(11)

mymodel_11_r2_0.6975_2022-06-03 19:06:51.535172-07:00.pkl
RandomForestRegressor(max_depth=14, max_features=0.4675944548647988,
                      min_samples_leaf=53, n_jobs=-1, verbose=1)
Full model r-squared scores: 0.79


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
pickled_model_reduced_11 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_11_r2_0.6889_2022-06-08 13:01:56.027263-07:00.pkl', 'rb'))
print(pickled_model_reduced_11)

RandomForestRegressor(max_depth=14, max_features=0.4675944548647988,
                      min_samples_leaf=53, n_jobs=-1, verbose=1)


In [ ]:
rf_var = pickled_model_reduced_11.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_11
predictions = rf.predict(x_test(11)) 
r2_test = r2_score(y_test(11)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

Reduced r-squared scores: 0.8


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
df_11 = pd.DataFrame({'Observed':y_test(11), 'Predicted':predictions})
compare_plot(df_11, 'Observed', 'Predicted')

December 2021

In [ ]:
testing_full_model(12)

mymodel_12_r2_0.8273_2022-06-03 17:08:24.774961-07:00.pkl
RandomForestRegressor(max_depth=16, max_features=0.4760312901950148,
                      min_samples_leaf=52, n_jobs=-1, verbose=1)
Full model r-squared scores: 0.6


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
#reduced_model_Dec
pickled_model_reduced_12 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_12_r2_0.7918_2022-06-03 17:24:47.590641-07:00.pkl', 'rb'))
print(pickled_model_reduced_12)

RandomForestRegressor(max_depth=16, max_features=0.4760312901950148,
                      min_samples_leaf=52, n_jobs=-1, verbose=1)


In [ ]:
rf_var = pickled_model_reduced_12.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_12
predictions = rf.predict(x_test(12)) 
r2_test = r2_score(y_test(12)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))


Reduced r-squared scores: 0.62


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
df = pd.DataFrame({'Observed':y_test(12), 'Predicted':predictions})
compare_plot(df, 'Observed', 'Predicted')

January 2022

In [ ]:
testing_full_model(1)

mymodel_1_r2_0.811_2022-06-03 19:14:41.271005-07:00.pkl
RandomForestRegressor(max_depth=16, max_features=0.397658127106747,
                      min_samples_leaf=53, n_jobs=-1, verbose=1)
Full model r-squared scores: 0.71


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
#reduced_model_Jan
pickled_model_reduced_1 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_1_r2_0.8108_2022-06-03 19:14:32.837441-07:00.pkl', 'rb'))
print(pickled_model_reduced_1)

RandomForestRegressor(max_depth=16, max_features=0.397658127106747,
                      min_samples_leaf=53, n_jobs=-1, verbose=1)


In [ ]:
rf_var = pickled_model_reduced_1.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_1
predictions = rf.predict(x_test(1)) 
r2_test = r2_score(y_test(1)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

Reduced r-squared scores: 0.69


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
df_1 = pd.DataFrame({'Observed':y_test(1), 'Predicted':predictions})
compare_plot(df_1, 'Observed', 'Predicted')

February 2022

In [ ]:
x_test(1).head(2)

,month,wb0,p0,p3sum,PPTAVG_BASIN,p6sum,WD_MAR_BASIN,pmpe,HLR19,WDMAX_BASIN,...,t1,PET_CAT,SILTAVE,RH_BASIN,RH_CAT,ELEV_MIN_M_BASIN_30M,wb1,KO_pct,TOPWET,SNOW_PCT_PRECIP
165,1,0.616038,5.342197,124.396754,42.4,160.969756,5.03,-307.47,0.0,5.17,...,7.950152,949.58,23.05,42.53,41.87,317.80,5.485696,2.74,9.25,3.82
166,1,31.850933,3.651251,241.540409,60.8,289.680167,6.80,-85.40,0.0,5.85,...,5.212169,785.32,32.03,42.81,41.09,595.97,70.941607,2.47,9.03,13.09


In [ ]:
x_test(2).head(2)

,month,wb0,p0,p3sum,PPTAVG_BASIN,p6sum,WD_MAR_BASIN,pmpe,HLR19,WDMAX_BASIN,...,t1,PET_CAT,SILTAVE,RH_BASIN,RH_CAT,ELEV_MIN_M_BASIN_30M,wb1,KO_pct,TOPWET,SNOW_PCT_PRECIP
247,2,1.099442,20.277867,104.192747,42.4,146.968666,5.03,-307.47,0.0,5.17,...,8.533094,949.58,23.05,42.53,41.87,317.80,0.616038,2.74,9.25,3.82
248,2,16.561921,17.535730,216.936287,60.8,262.792849,6.80,-85.40,0.0,5.85,...,8.001316,785.32,32.03,42.81,41.09,595.97,31.850933,2.47,9.03,13.09


In [ ]:
testing_full_model(2)

mymodel_2_r2_0.6867_2022-06-07 18:21:49.151868-07:00.pkl
RandomForestRegressor(max_depth=19, max_features=0.3187382635730718,
                      min_samples_leaf=52, n_estimators=250, n_jobs=-1,
                      verbose=1)
mymodel_2_r2_0.8122_2022-06-22 15:04:16.713044-07:00.pkl
RandomForestRegressor(max_depth=19, max_features=0.3187382635730718,
                      min_samples_leaf=52, n_estimators=250, n_jobs=-1,
                      verbose=1)
Full model r-squared scores: -2.63


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
pickled_model_reduced_2 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_2_r2_0.6936_2022-06-07 18:21:33.164904-07:00.pkl', 'rb'))
print(pickled_model_reduced_2)

RandomForestRegressor(max_depth=19, max_features=0.3187382635730718,
                      min_samples_leaf=52, n_estimators=250, n_jobs=-1,
                      verbose=1)


In [ ]:
rf_var = pickled_model_reduced_2.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_2
predictions = rf.predict(x_test(2)) 
r2_test = r2_score(y_test(2)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

Reduced r-squared scores: -2.33


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
df_2_1 = pd.DataFrame({'Observed':y_test(2), 'Predicted':predictions})
compare_plot(df_2_1, 'Observed', 'Predicted')

In [ ]:
pickled_model_reduced_2_second = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_2_r2_0.8142_2022-06-22 15:04:01.221627-07:00.pkl', 'rb'))
print(pickled_model_reduced_2_second)

RandomForestRegressor(max_depth=19, max_features=0.3187382635730718,
                      min_samples_leaf=52, n_estimators=250, n_jobs=-1,
                      verbose=1)


In [ ]:
rf_var = pickled_model_reduced_2_second.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_2_second
predictions = rf.predict(x_test(2)) 
r2_test = r2_score(y_test(2)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

Reduced r-squared scores: -2.75


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
df_2_2 = pd.DataFrame({'Observed':y_test(2), 'Predicted':predictions})
compare_plot(df_2_2, 'Observed', 'Predicted')

March 2022

In [ ]:
testing_full_model(3)

mymodel_3_r2_0.7557_2022-06-07 19:33:44.365260-07:00.pkl
RandomForestRegressor(max_depth=10, max_features=0.341540726788639,
                      min_samples_leaf=50, n_estimators=500, n_jobs=-1,
                      verbose=1)


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Full model r-squared scores: 0.46


In [ ]:
pickled_model_reduced_3 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_3_r2_0.7541_2022-06-07 19:33:13.366067-07:00.pkl', 'rb'))
print(pickled_model_reduced_3)

RandomForestRegressor(max_depth=10, max_features=0.341540726788639,
                      min_samples_leaf=50, n_estimators=500, n_jobs=-1,
                      verbose=1)


In [ ]:
rf_var = pickled_model_reduced_3.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_3
predictions = rf.predict(x_test(3)) 
r2_test = r2_score(y_test(3)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Reduced r-squared scores: 0.27


In [ ]:
df_3 = pd.DataFrame({'Observed':y_test(3), 'Predicted':predictions})
compare_plot(df_3, 'Observed', 'Predicted')

April 2022

In [ ]:
testing_full_model(4)

mymodel_4_r2_0.7271_2022-06-08 01:04:44.388674-07:00.pkl
RandomForestRegressor(max_depth=11, max_features=0.4055057735086026,
                      min_samples_leaf=55, n_estimators=250, n_jobs=-1,
                      verbose=1)
Full model r-squared scores: 0.83


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
pickled_model_reduced_4 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/Retrain_RF_reduced_features/mymodel_reduced_4_r2_0.7253_2022-06-08 01:04:25.772044-07:00.pkl', 'rb'))
print(pickled_model_reduced_4)

RandomForestRegressor(max_depth=11, max_features=0.4055057735086026,
                      min_samples_leaf=55, n_estimators=250, n_jobs=-1,
                      verbose=1)


In [ ]:
rf_var = pickled_model_reduced_4.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_reduced_4
predictions = rf.predict(x_test(4)) 
r2_test = r2_score(y_test(4)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

Reduced r-squared scores: 0.83


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
df_4 = pd.DataFrame({'Observed':y_test(4), 'Predicted':predictions})
compare_plot(df_4, 'Observed', 'Predicted')